In [1]:
# Import necessary modules
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import sys

here = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.abspath(os.path.join(here, '../../'))
sys.path.insert(0, project_root)

from core.ga.ga_bounds import DesignSpaceBounds
from core.ga.ga_params import GeneticAlgorithmParams
from core.ga.ga_run import GeneticAlgorithm

from core.model.model_carrying_capacities import ModelCarryingCapacities
from core.model.model_disturbances import ModelDisturbances
from core.model.model_growth_rates import ModelGrowthRates
from core.model.model_initial_conditions import ModelInitialConditions
from core.model.model_params import ModelParams
from core.model.model_typical_disturbances import ModelTypicalDisturbances

In [2]:
input_key = 1

## Model Parameters

In [3]:
# Set ModelParams (time stepping)
model_params = ModelParams(
    dt = 0.1, # hours/step
    simulation_hours = 2900 # hours
)

In [4]:
# Set ModelCarryingCapacities
carrying_capacities = ModelCarryingCapacities(
    kh = 3.0,  # m
    kA = 0.65, # m2
    kN = 20,   # number of leaves
    kc = 1000, # number of spikelets
    kP = 0.25  # kg
)

In [5]:
# Set ModelGrowthRates
growth_rates = ModelGrowthRates(
    ah = 0.01,   # 1/hr
    aA = 0.0105, # 1/hr
    aN = 0.011,  # 1/hr
    ac = 0.01,   # 1/hr
    aP = 0.005   # 1/hr
)

In [6]:
# Set ModelInitialConditions
initial_conditions = ModelInitialConditions(
    h0=carrying_capacities.kh/model_params.simulation_hours, # m/hr
    A0=carrying_capacities.kA/model_params.simulation_hours, # m2/hr
    N0=carrying_capacities.kN/model_params.simulation_hours, # number/hr
    c0=carrying_capacities.kc/model_params.simulation_hours, # number/hr
    P0=carrying_capacities.kP/model_params.simulation_hours  # kg/hr
)

## Design Space

In [7]:
lower_bounds_dict = {
    "irrigation_frequency":    24,     # once every X hours
    "irrigation_amount":       0.001, # inches per irrigation
    "fertilization_frequency": 24,    # once every X hours (once a day)
    "fertilization_amount":    0.1  # pounds per fertilization
}
lower_bounds = np.array([value for key, value in lower_bounds_dict.items()])

In [8]:
upper_bounds_dict = {
    "irrigation_frequency":    24 * 7 * 10,   # once every 10 weeks
    "irrigation_amount":       10,            # inches per irrigation
    "fertilization_frequency": 30 * 24 * 5,  # once every 5 months
    "fertilization_amount":    300             # pounds per fertilization
}
upper_bounds = np.array([value for key, value in upper_bounds_dict.items()])

In [9]:
# Set DesignSpaceBounds
bounds = DesignSpaceBounds(
    lower_bounds=lower_bounds,
    upper_bounds=upper_bounds
)

## Genetic Algorithm Parameters

In [10]:
# Genetic algorithm style inputs
ga_params = GeneticAlgorithmParams(
    num_parents     = 16,  # number of design strings to breed
    num_kids        = 16,  # number of offspring design strings
    num_generations = 100, # number of generations
    num_members     = 128, # number of simulations per generation
)

## Input Disturbances

In [11]:
# Hourly precipitation, radiation, and temperature from CSV
hourly_input_disturbances = pd.read_csv(
    '../../io/inputs/hourly_prcp_rad_temp_iowa.csv'
)

In [12]:
# Set ModelDisturbances
input_disturbances = ModelDisturbances(
    precipitation = hourly_input_disturbances['Hourly Precipitation (in)'].to_numpy(),
    radiation     = hourly_input_disturbances['Hourly Radiation (W/m2)'].to_numpy(),
    temperature   = hourly_input_disturbances['Temperature (C)'].to_numpy()
)

## Optimal/Typical Values

In [13]:
typical_disturbances = ModelTypicalDisturbances(
        optimal_cumulative_water      = 28,  # inches/acre per season
        optimal_cumulative_fertilizer = 355, # lbs/acre per season
        typical_temperature           = np.mean(input_disturbances.temperature),   # typical temperature over the season
        typical_radiation             = 3/4 * np.max(input_disturbances.radiation) # typical radiation over the season
    )

## Genetic Algorithm

In [14]:
# Create GeneticAlgorithm object from all inputs
GeneticAlgorithm = GeneticAlgorithm(
    bounds               = bounds,
    ga_params            = ga_params,
    model_params         = model_params,
    carrying_capacities  = carrying_capacities,
    growth_rates         = growth_rates,
    initial_conditions   = initial_conditions,
    disturbances         = input_disturbances,
    typical_disturbances = typical_disturbances
)

In [15]:
# Run the Genetic Algorithm and save the result

GeneticAlgorithmResult = GeneticAlgorithm.run_with_lambda()

with open(f"ga_parallel_results/pickled_ga_results/ga_parallel_result_{input_key}.pkl", "wb") as f:
    pickle.dump(GeneticAlgorithmResult, f)


Time taken to calculate costs for g=0: 2.691655158996582 seconds
Generation 2 of 100
Lowest cost in generation 1: -338.97
Generation 3 of 100


KeyboardInterrupt: 

In [ ]:
with open(f"ga_parallel_results/pickled_ga_results/ga_parallel_result_{input_key}.pkl", "rb") as f:
    result = pickle.load(f)

In [ ]:
[unique_values, unique_costs] = result.final_population.get_unique_designs()

In [ ]:
unique_values_df = pd.DataFrame({
    'Irrigation frequency (every X hours)': unique_values[:, 0],
    'Irrigation amount (in)':               unique_values[:, 1],
    'Fertilizer frequency (every X hours)': unique_values[:, 2],
    'Fertilizer amount (lb)':               unique_values[:, 3],
    'Cost ($)':                             unique_costs,
    'Revenue ($)':                          -unique_costs,
})
unique_values_df.to_csv(f'ga_parallel_results/unique_values_costs_results/ga_parallel_result_unique_values_and_costs_{input_key}.csv', index=False)

In [ ]:
min(unique_costs)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

member_ids = np.arange(unique_costs.shape[0])
ax.plot(member_ids, unique_costs)

ax.set_title("Costs across final population", fontsize=16, pad=15)
ax.set_xlabel("Member", fontsize=13)
ax.set_ylabel("Cost", fontsize=13)

fig.tight_layout()

In [ ]:
result.plot_optimization_results()

In [ ]:
unique_values[0]